In [1]:
import pandas as pd
import numpy as np
import os
from typing import Callable # this is to define test_func functions dtype as function itself
import re # for regural expression
import warnings
warnings.filterwarnings('ignore')
import pickle # for pickling my model

In [2]:
df = pd.read_csv(r'/kaggle/input/mobile-recommendation-system-dataset/mobile_recommendation_system_dataset.csv',index_col=False)

In [3]:
def random_values(col_name:str,total=6):
    for i in range(0,total):
        index = np.random.randint(df.shape[0])
        print(index,':',df[col_name].iloc[index])        

In [4]:
random_values('corpus')

1067 : Storage64 GBRAM4 GBExpandable Storage1TB  SystemAndroid 12Processor TypeMediatek Helio G37Processor Speed2.3 50MP 2MP 50MP 2MP 8MP 8MP 4G Capacity5000 Display Size16.51 cm (6.5 inch)Resolution1600 x 720 PixelsResolution TypeHD+GPUIMG PowerVR GE8320Display TypeHD+ IPS LCD DisplayOther Display FeaturesRefresh Rate: 90 Hz, Aspect Ratio: 20:9, Screen-to-Body Ratio: 89.03%
769 : Storage128 GBRAM6  SystemAndroid 11Processor Speed2 64MP 4G Capacity6000 Display Size16.26 cm (6.4 inch)Resolution2400 x 1080 Pixels
2199 : Storage128 GBRAM6 GBExpandable Storage1TB  SystemAndroid 11Processor TypeMediatek Helio G85Processor Speed2 50MP 8MP 2MP 13MP 4G Capacity5000 Display Size16.26 cm (6.4 inch)Resolution2400 x 1080 PixelsResolution TypeFull HD+GPUARM Mali-G52 MC2Display TypeFHD+ AMOLED DisplayOther Display Features60 Hz Refresh Rate, 20:9 Display Aspect Ratio, 88.80% Screen-to-Body Ratio, Upto 700nits Brightness
929 : Storage256  SystemiOS 16Processor TypeA16 Bionic Chip 48MP 12MP 12MP 48MP 

## Lowercasing the corpus values

In [5]:
df['corpus'] = df['corpus'].str.lower()

In [6]:
random_values('corpus')

738 : storage64 gbram4 gbexpandable storage256gb  systemandroid oreo 8.1processor typequalcomm snapdragon 626processor speed2.2 16mp 5mp 24mp 3g capacity3260 display size16.0 cm (6.3 inch)resolution2280 x 1080 pixelsresolution typefull hd+gpuadreno 506display typeips capacitive touch screenother display featuresfullview display 2.0, 1.75 mm ultra-slim bezel, 90% screen-to-body ratio, face scan to unlock, face scan for app-lock, smart motion, corning gorilla 2.5d glass , eye protection mode, smart split 3.0, 19:9 display
215 : storage128 gbram6  systemandroid q 12processor speed2.2 50mp 4g capacity6000 display size16.76 cm (6.6 inch)resolution720x2408 pixel
1230 : storage128 gbram6 gbexpandable storage1tb  systemandroid 11processor typequalcomm snapdragon 662processor speed2 48mp 2mp 2mp 48mp 2mp 2mp 48mp 2mp 2mp 48mp 16mp 16mp 4g capacity5000 display size16.71 cm (6.58 inch)resolution2408 x 1080 pixelsresolution typefull hd+gpuadreno 610display typefull hd+ lcd full-incell ips displayo

In [7]:
df.isnull().sum()

name        0
ratings     0
price       0
imgURL      0
corpus     12
dtype: int64

In [8]:
df.dropna(inplace=True)

In [9]:
max_len = 0
for i in range(0, len(df['corpus'])):
    try:
        if len(df['corpus'][i]) > max_len:
            max_len = len(df['corpus'][i])
    except:
        pass

print(f'words max len in corpus is: {max_len}')


words max len in corpus is: 1196


In [10]:
df.sample(10)

,name,ratings,price,imgURL,corpus
500,"Infinix Smart 7 HD (Green Apple, 64 GB)",4.3,6199,https://rukminim2.flixcart.com/image/312/312/x...,storage64 systemandroid 12processor typespread...
2035,"APPLE iPhone 6 (Silver, 16 GB)",4.4,28546,https://rukminim2.flixcart.com/image/312/312/m...,storage16 systemios 8processor typeapple a8 64...
589,"realme 10 (Rush Black, 128 GB)",4.3,16999,https://rukminim2.flixcart.com/image/312/312/x...,storage128 gbram8 gbexpandable storage1tb sys...
977,"APPLE iPhone 6 (Grey, 128 GB)",4.6,5964,https://rukminim2.flixcart.com/image/312/312/m...,storage128 systemios 8processor typeapple a8 6...
444,"Tecno Pova Neo (Obsidian Black, 128 GB)",4.1,10999,https://rukminim2.flixcart.com/image/312/312/k...,storage128 gbram6 gbexpandable storage256gb s...
1332,"realme 2 Pro (Ice Lake, 64 GB)",4.5,13990,https://rukminim2.flixcart.com/image/312/312/j...,storage64 gbram4 gbexpandable storage256gb sy...
848,"APPLE iPhone 7 (Red, 128 GB)",4.1,30636,https://rukminim2.flixcart.com/image/312/312/j...,storage128 systemios 12processor typeapple a10...
1840,"OPPO A55 (Rainbow Blue, 64 GB)",4.3,"₹18,990",https://rukminim2.flixcart.com/image/312/312/k...,storage64 gbram4 gbexpandable storage256gb sy...
56,"REDMI 11 Prime 5G (Meadow Green, 128 GB)",4.2,15790,https://rukminim2.flixcart.com/image/312/312/x...,storage128 gbram6 systemandroid 12primary clo...
148,"Xiaomi 12 Pro 5G (Couture Blue, 256 GB)",4.1,23537,https://rukminim2.flixcart.com/image/312/312/x...,storage256 gbram8 systemandroid 12processor t...


In [11]:
random_values('imgURL')

823 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/9/l/3/-original-imaghgcktmwxthzc.jpeg?q=70
2049 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/2/s/7/-original-imagmg6gktts6sfy.jpeg?q=70
1790 : https://rukminim2.flixcart.com/image/312/312/ktketu80/mobile/a/i/l/iphone-13-mini-mlk93hn-a-apple-original-imag6vp6x67yypzk.jpeg?q=70
2475 : https://rukminim2.flixcart.com/image/312/312/l3rmzrk0/mobile/2/l/k/-original-imagetmebhg9thpv.jpeg?q=70
420 : https://rukminim2.flixcart.com/image/312/312/k2jbyq80pkrrdj/mobile-refurbished/x/j/s/iphone-11-128-d-mwm02hn-a-apple-0-original-imafkg242ugz8hwc.jpeg?q=70
817 : https://rukminim2.flixcart.com/image/312/312/ktketu80/mobile/r/8/l/iphone-13-mini-mlk53hn-a-apple-original-imag6vp693zncyk7.jpeg?q=70


# Model

## vectorization(to convert the words to vectors)

In [12]:

from sklearn.feature_extraction.text import CountVectorizer 

In [13]:
cont_vect = CountVectorizer(max_features=1000)

In [14]:
arrays = cont_vect.fit_transform(df['corpus'])

In [15]:
vectors = arrays.toarray()

In [16]:
cont_vect.get_feature_names_out()

array(['00', '000', '02', '03', '04', '05', '07', '07bother', '08', '09',
       '0mp', '0primary', '0processor', '10', '100', '1000', '100000',
       '1000000', '100mp', '103', '104', '106', '1080', '1080processor',
       '1080resolution', '108mp', '10mp', '10processor', '11', '1100',
       '1125', '1170', '1179', '11primary', '11processor', '12', '120',
       '1200', '1200processor', '120hz', '1242', '1280processor', '1284',
       '1290', '12mp', '12primary', '12processor', '13', '1300',
       '1300processor', '1330', '1334', '135', '13mp', '13processor',
       '14', '1400', '144', '1440', '144hz', '1480', '14processor', '15',
       '1500', '1544', '15processor', '16', '160', '1600', '16000',
       '1600resolution', '1612', '16mp', '16processor', '18', '180',
       '1800', '180hz', '19', '1920', '1920hz', '1processor', '20', '200',
       '2000000', '200mp', '20mp', '21', '22', '23', '2340', '24', '240',
       '2400', '2400resolution', '2408', '240hz', '2460', '24mp', '25m

## Calculating cosine similarity for recommendation

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
similarity = cosine_similarity(vectors)
similarity.shape # this is because I have calculated the similarity among all the rows to all the rows(confusing?? but try to understand)

(2534, 2534)

In [19]:
similarity[3]

array([0.3345748 , 0.17485208, 0.53136893, ..., 0.57564968, 0.27586342,
       0.37283655])

In [20]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:11]

[(68, 0.9999999999999997),
 (303, 0.9999999999999997),
 (616, 0.9850746268656713),
 (1461, 0.9850746268656713),
 (457, 0.970149253731343),
 (1027, 0.970149253731343),
 (1735, 0.970149253731343),
 (569, 0.9468131937613656),
 (1123, 0.9468131937613656),
 (1540, 0.9468131937613656)]

In [21]:
def recommend(mobile):
    mobile_index = df[df['name']==mobile].index[0]
    similarity_array = similarity[mobile_index]
    similar_10_mobiles = sorted(list(enumerate(similarity_array)),reverse=True,key=lambda x:x[1])[1:11]
    
    for i in similar_10_mobiles:
        print(df['name'].iloc[i[0]])



In [22]:
recommend('APPLE iPhone 13 mini (Blue, 128 GB)')

APPLE iPhone 13 mini ((PRODUCT)RED, 128 GB)
APPLE iPhone 13 mini (Starlight, 128 GB)
APPLE iPhone 13 mini (Green, 128 GB)
APPLE iPhone 13 mini (Midnight, 128 GB)
APPLE iPhone 13 mini (Green, 256 GB)
APPLE iPhone 13 mini (Midnight, 512 GB)
APPLE iPhone 13 mini (Pink, 512 GB)
APPLE iPhone 13 mini (Starlight, 256 GB)
APPLE iPhone 13 mini (Midnight, 256 GB)
APPLE iPhone 13 mini (Pink, 256 GB)


## making Pickle file of df and similarities to use in web

In [23]:
os.makedirs(r'src\model',exist_ok=True)

In [24]:
pickle.dump(obj=df,file=open(file=r'src\model\dataframe.pkl',mode='wb'))

In [25]:
pickle.dump(obj=similarity,file=open(file=r'src\model\similarity.pkl',mode='wb'))